## 1. Import Libraries and Setup

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scikit-learn utilities
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Surprise library for collaborative filtering
from surprise import Dataset, Reader, SVD, KNNBasic, KNNWithMeans
from surprise.model_selection import cross_validate, train_test_split as surprise_train_test_split
from surprise import accuracy

# Scipy for sparse matrices
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

# Date handling
from datetime import datetime

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# Random seed
np.random.seed(42)

print("Libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Load datasets
movies = pd.read_csv('../../../data/data/movies.csv')
ratings = pd.read_csv('../../../data/data/ratings.csv')

print("Movies Dataset:")
print(f"Shape: {movies.shape}")
print(movies.head())

print("\n" + "="*80)
print("\nRatings Dataset:")
print(f"Shape: {ratings.shape}")
print(ratings.head())

In [ ]:
# Data information
print("Movies Info:")
print(movies.info())
print("\n" + "="*80)
print("\nRatings Info:")
print(ratings.info())

print("\n" + "="*80)
print("\nBasic Statistics:")
print(f"Total movies: {len(movies):,}")
print(f"Total ratings: {len(ratings):,}")
print(f"Unique users: {ratings['userId'].nunique():,}")
print(f"Unique movies rated: {ratings['movieId'].nunique():,}")
print(f"Rating range: {ratings['rating'].min()} - {ratings['rating'].max()}")
print(f"Average rating: {ratings['rating'].mean():.2f}")
print(f"Sparsity: {(1 - len(ratings) / (ratings['userId'].nunique() * ratings['movieId'].nunique())) * 100:.2f}%")

In [ ]:
# Check for missing values
print("Missing Values - Movies:")
print(movies.isnull().sum())
print("\nMissing Values - Ratings:")
print(ratings.isnull().sum())

## 3. Exploratory Data Analysis

In [ ]:
# Rating distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram of ratings
axes[0, 0].hist(ratings['rating'], bins=10, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Rating', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Distribution of Ratings', fontsize=14, fontweight='bold')
axes[0, 0].axvline(ratings['rating'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {ratings["rating"].mean():.2f}')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# Ratings per user
ratings_per_user = ratings.groupby('userId').size()
axes[0, 1].hist(ratings_per_user, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Number of Ratings', fontsize=12)
axes[0, 1].set_ylabel('Number of Users', fontsize=12)
axes[0, 1].set_title('Ratings per User Distribution', fontsize=14, fontweight='bold')
axes[0, 1].axvline(ratings_per_user.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {ratings_per_user.mean():.1f}')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# Ratings per movie
ratings_per_movie = ratings.groupby('movieId').size()
axes[1, 0].hist(ratings_per_movie, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 0].set_xlabel('Number of Ratings', fontsize=12)
axes[1, 0].set_ylabel('Number of Movies', fontsize=12)
axes[1, 0].set_title('Ratings per Movie Distribution', fontsize=14, fontweight='bold')
axes[1, 0].axvline(ratings_per_movie.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {ratings_per_movie.mean():.1f}')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].set_yscale('log')

# Value counts of ratings
rating_counts = ratings['rating'].value_counts().sort_index()
axes[1, 1].bar(rating_counts.index, rating_counts.values, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].set_xlabel('Rating Value', fontsize=12)
axes[1, 1].set_ylabel('Count', fontsize=12)
axes[1, 1].set_title('Rating Value Counts', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Top rated movies (with minimum number of ratings)
min_ratings = 50
movie_stats = ratings.groupby('movieId').agg({
    'rating': ['mean', 'count']
}).reset_index()
movie_stats.columns = ['movieId', 'avg_rating', 'num_ratings']

# Merge with movie titles
movie_stats = movie_stats.merge(movies[['movieId', 'title']], on='movieId')
popular_movies = movie_stats[movie_stats['num_ratings'] >= min_ratings].sort_values('avg_rating', ascending=False)

print(f"Top 15 Movies (with at least {min_ratings} ratings):")
print(popular_movies.head(15).to_string(index=False))

In [ ]:
# Genre analysis
# Extract genres
movies['genres_list'] = movies['genres'].str.split('|')
all_genres = []
for genres in movies['genres_list']:
    if isinstance(genres, list):
        all_genres.extend(genres)

genre_counts = pd.Series(all_genres).value_counts()
genre_counts = genre_counts[genre_counts.index != '(no genres listed)']

# Plot genre distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Bar chart
genre_counts.plot(kind='barh', ax=ax1, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Number of Movies', fontsize=12)
ax1.set_ylabel('Genre', fontsize=12)
ax1.set_title('Movie Count by Genre', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Average rating by genre
genre_ratings = []
for genre in genre_counts.index:
    genre_mask = movies['genres'].str.contains(genre, na=False)
    genre_movie_ids = movies[genre_mask]['movieId']
    genre_avg_rating = ratings[ratings['movieId'].isin(genre_movie_ids)]['rating'].mean()
    genre_ratings.append(genre_avg_rating)

genre_rating_df = pd.DataFrame({
    'Genre': genre_counts.index,
    'Avg_Rating': genre_ratings
}).sort_values('Avg_Rating', ascending=True)

ax2.barh(genre_rating_df['Genre'], genre_rating_df['Avg_Rating'], edgecolor='black', alpha=0.7, color='orange')
ax2.set_xlabel('Average Rating', fontsize=12)
ax2.set_ylabel('Genre', fontsize=12)
ax2.set_title('Average Rating by Genre', fontsize=14, fontweight='bold')
ax2.axvline(ratings['rating'].mean(), color='red', linestyle='--', linewidth=2, label='Overall Avg')
ax2.legend()
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Data Preparation

In [ ]:
# Filter users and movies with minimum ratings
min_user_ratings = 20
min_movie_ratings = 10

# Count ratings per user and movie
user_counts = ratings['userId'].value_counts()
movie_counts = ratings['movieId'].value_counts()

# Filter
active_users = user_counts[user_counts >= min_user_ratings].index
popular_movies_ids = movie_counts[movie_counts >= min_movie_ratings].index

ratings_filtered = ratings[
    (ratings['userId'].isin(active_users)) & 
    (ratings['movieId'].isin(popular_movies_ids))
].copy()

print(f"Original ratings: {len(ratings):,}")
print(f"Filtered ratings: {len(ratings_filtered):,} ({len(ratings_filtered)/len(ratings)*100:.1f}%)")
print(f"Unique users: {ratings_filtered['userId'].nunique():,}")
print(f"Unique movies: {ratings_filtered['movieId'].nunique():,}")

In [ ]:
# Create user-item matrix
user_item_matrix = ratings_filtered.pivot(index='userId', columns='movieId', values='rating')
print(f"User-Item Matrix Shape: {user_item_matrix.shape}")
print(f"Sparsity: {(user_item_matrix.isna().sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1])) * 100:.2f}%")
print("\nSample of User-Item Matrix:")
print(user_item_matrix.iloc[:5, :5])

In [ ]:
# Train-test split
train_data, test_data = train_test_split(ratings_filtered, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_data):,}")
print(f"Test set size: {len(test_data):,}")
print(f"Split ratio: {len(train_data)/len(ratings_filtered)*100:.1f}% / {len(test_data)/len(ratings_filtered)*100:.1f}%")

## 5. User-Based Collaborative Filtering

In [ ]:
# Prepare data for Surprise
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))
data = Dataset.load_from_df(ratings_filtered[['userId', 'movieId', 'rating']], reader)

# Split data
trainset, testset = surprise_train_test_split(data, test_size=0.2, random_state=42)

print("Data prepared for Surprise library")
print(f"Training set size: {trainset.n_ratings:,}")
print(f"Test set size: {len(testset):,}")

In [ ]:
# User-based collaborative filtering
print("Training User-Based Collaborative Filtering...")
user_based = KNNWithMeans(k=40, sim_options={
    'name': 'cosine',
    'user_based': True
})

user_based.fit(trainset)

# Predict on test set
predictions_user = user_based.test(testset)

# Calculate metrics
rmse_user = accuracy.rmse(predictions_user, verbose=False)
mae_user = accuracy.mae(predictions_user, verbose=False)

print(f"\nUser-Based Collaborative Filtering Results:")
print(f"RMSE: {rmse_user:.4f}")
print(f"MAE: {mae_user:.4f}")

## 6. Item-Based Collaborative Filtering

In [ ]:
# Item-based collaborative filtering
print("Training Item-Based Collaborative Filtering...")
item_based = KNNWithMeans(k=40, sim_options={
    'name': 'cosine',
    'user_based': False
})

item_based.fit(trainset)

# Predict on test set
predictions_item = item_based.test(testset)

# Calculate metrics
rmse_item = accuracy.rmse(predictions_item, verbose=False)
mae_item = accuracy.mae(predictions_item, verbose=False)

print(f"\nItem-Based Collaborative Filtering Results:")
print(f"RMSE: {rmse_item:.4f}")
print(f"MAE: {mae_item:.4f}")

## 7. Matrix Factorization (SVD)

In [ ]:
# SVD (Singular Value Decomposition)
print("Training SVD Model...")
svd_model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
svd_model.fit(trainset)

# Predict on test set
predictions_svd = svd_model.test(testset)

# Calculate metrics
rmse_svd = accuracy.rmse(predictions_svd, verbose=False)
mae_svd = accuracy.mae(predictions_svd, verbose=False)

print(f"\nSVD Model Results:")
print(f"RMSE: {rmse_svd:.4f}")
print(f"MAE: {mae_svd:.4f}")

In [ ]:
# Cross-validation for SVD
print("\nPerforming 5-Fold Cross-Validation for SVD...")
cv_results = cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)

print(f"\nCross-Validation Results:")
print(f"RMSE: {cv_results['test_rmse'].mean():.4f} (+/- {cv_results['test_rmse'].std():.4f})")
print(f"MAE: {cv_results['test_mae'].mean():.4f} (+/- {cv_results['test_mae'].std():.4f})")

## 8. Content-Based Filtering

In [ ]:
# Create content features from genres
movies_content = movies.copy()
movies_content['genres_clean'] = movies_content['genres'].str.replace('|', ' ')

# TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_content['genres_clean'].fillna(''))

print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"Features: {tfidf.get_feature_names_out()}")

In [ ]:
# Calculate cosine similarity between movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(f"Cosine Similarity Matrix Shape: {cosine_sim.shape}")

# Create indices mapping
indices = pd.Series(movies_content.index, index=movies_content['movieId']).to_dict()

def get_content_based_recommendations(movie_id, top_n=10):
    """
    Get content-based recommendations for a given movie.
    """
    if movie_id not in indices:
        return pd.DataFrame()
    
    idx = indices[movie_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # Exclude the movie itself
    
    movie_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]
    
    recommendations = movies_content.iloc[movie_indices][['movieId', 'title', 'genres']].copy()
    recommendations['similarity'] = similarity_scores
    
    return recommendations

# Test content-based recommendations
test_movie_id = ratings_filtered['movieId'].iloc[0]
test_movie = movies[movies['movieId'] == test_movie_id].iloc[0]

print(f"\nContent-Based Recommendations for: {test_movie['title']} ({test_movie['genres']})")
print("="*80)
content_recs = get_content_based_recommendations(test_movie_id, top_n=10)
print(content_recs.to_string(index=False))

## 9. Hybrid Recommendation System

In [ ]:
def get_hybrid_recommendations(user_id, top_n=10, alpha=0.7):
    """
    Hybrid recommendation combining collaborative and content-based filtering.
    
    Parameters:
    - user_id: User ID
    - top_n: Number of recommendations
    - alpha: Weight for collaborative filtering (1-alpha for content-based)
    """
    # Get user's rated movies
    user_ratings = ratings_filtered[ratings_filtered['userId'] == user_id]
    rated_movie_ids = set(user_ratings['movieId'])
    
    # Get all movies not rated by user
    all_movie_ids = set(ratings_filtered['movieId'].unique())
    unrated_movie_ids = list(all_movie_ids - rated_movie_ids)
    
    # Collaborative filtering predictions
    cf_predictions = []
    for movie_id in unrated_movie_ids:
        pred = svd_model.predict(user_id, movie_id)
        cf_predictions.append((movie_id, pred.est))
    
    cf_df = pd.DataFrame(cf_predictions, columns=['movieId', 'cf_score'])
    
    # Content-based scores
    # Calculate average similarity to user's highly-rated movies
    high_rated = user_ratings[user_ratings['rating'] >= 4.0]
    
    content_scores = []
    for movie_id in unrated_movie_ids:
        if movie_id not in indices:
            content_scores.append((movie_id, 0))
            continue
        
        similarities = []
        for rated_id in high_rated['movieId']:
            if rated_id in indices:
                idx1 = indices[movie_id]
                idx2 = indices[rated_id]
                similarities.append(cosine_sim[idx1][idx2])
        
        avg_sim = np.mean(similarities) if similarities else 0
        content_scores.append((movie_id, avg_sim))
    
    content_df = pd.DataFrame(content_scores, columns=['movieId', 'content_score'])
    
    # Combine scores
    hybrid_df = cf_df.merge(content_df, on='movieId')
    
    # Normalize scores
    hybrid_df['cf_score_norm'] = (hybrid_df['cf_score'] - hybrid_df['cf_score'].min()) / \
                                  (hybrid_df['cf_score'].max() - hybrid_df['cf_score'].min())
    hybrid_df['content_score_norm'] = (hybrid_df['content_score'] - hybrid_df['content_score'].min()) / \
                                       (hybrid_df['content_score'].max() - hybrid_df['content_score'].min() + 1e-10)
    
    # Hybrid score
    hybrid_df['hybrid_score'] = alpha * hybrid_df['cf_score_norm'] + (1 - alpha) * hybrid_df['content_score_norm']
    
    # Get top recommendations
    top_recs = hybrid_df.nlargest(top_n, 'hybrid_score')
    
    # Merge with movie info
    recommendations = top_recs.merge(movies[['movieId', 'title', 'genres']], on='movieId')
    
    return recommendations[['movieId', 'title', 'genres', 'cf_score', 'content_score', 'hybrid_score']]

# Test hybrid recommendations
test_user = ratings_filtered['userId'].iloc[0]
print(f"\nHybrid Recommendations for User {test_user}:")
print("="*80)
hybrid_recs = get_hybrid_recommendations(test_user, top_n=15, alpha=0.7)
print(hybrid_recs.to_string(index=False))

## 10. Evaluation Metrics

In [ ]:
# Compare all methods
comparison_df = pd.DataFrame({
    'Method': ['User-Based CF', 'Item-Based CF', 'SVD (Matrix Factorization)'],
    'RMSE': [rmse_user, rmse_item, rmse_svd],
    'MAE': [mae_user, mae_item, mae_svd]
})

print("\nMODEL COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("\nNote: Lower RMSE and MAE indicate better performance.")

In [ ]:
# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# RMSE comparison
ax1.bar(comparison_df['Method'], comparison_df['RMSE'], edgecolor='black', alpha=0.7)
ax1.set_ylabel('RMSE', fontsize=12)
ax1.set_title('RMSE Comparison (Lower is Better)', fontsize=14, fontweight='bold')
ax1.set_xticklabels(comparison_df['Method'], rotation=15, ha='right')
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(comparison_df['RMSE']):
    ax1.text(i, v, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

# MAE comparison
ax2.bar(comparison_df['Method'], comparison_df['MAE'], edgecolor='black', alpha=0.7, color='orange')
ax2.set_ylabel('MAE', fontsize=12)
ax2.set_title('MAE Comparison (Lower is Better)', fontsize=14, fontweight='bold')
ax2.set_xticklabels(comparison_df['Method'], rotation=15, ha='right')
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate(comparison_df['MAE']):
    ax2.text(i, v, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Precision@K evaluation
def precision_at_k(predictions, k=10, threshold=4.0):
    """
    Calculate Precision@K for recommendation system.
    """
    # Group predictions by user
    user_est_true = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_est_true:
            user_est_true[uid] = []
        user_est_true[uid].append((est, true_r))
    
    precisions = []
    for uid, user_ratings in user_est_true.items():
        # Sort by estimated rating
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Get top k
        top_k = user_ratings[:k]
        
        # Count relevant items (actual rating >= threshold)
        n_rel = sum(1 for (_, true_r) in top_k if true_r >= threshold)
        
        # Precision = relevant / k
        precisions.append(n_rel / k)
    
    return np.mean(precisions)

# Calculate Precision@K for different k values
k_values = [5, 10, 15, 20]
precision_results = []

for method, predictions in [('User-Based CF', predictions_user), 
                            ('Item-Based CF', predictions_item),
                            ('SVD', predictions_svd)]:
    method_precisions = []
    for k in k_values:
        prec = precision_at_k(predictions, k=k, threshold=4.0)
        method_precisions.append(prec)
    precision_results.append(method_precisions)

# Plot Precision@K
plt.figure(figsize=(12, 7))
for idx, method in enumerate(['User-Based CF', 'Item-Based CF', 'SVD']):
    plt.plot(k_values, precision_results[idx], marker='o', linewidth=2, markersize=8, label=method)

plt.xlabel('K (Number of Recommendations)', fontsize=12)
plt.ylabel('Precision@K', fontsize=12)
plt.title('Precision@K Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nPrecision@K Results:")
print("="*80)
for idx, method in enumerate(['User-Based CF', 'Item-Based CF', 'SVD']):
    print(f"\n{method}:")
    for k, prec in zip(k_values, precision_results[idx]):
        print(f"  Precision@{k}: {prec:.4f}")

## 11. Generate Recommendations for Sample Users

In [ ]:
# Sample users for demonstration
sample_users = ratings_filtered['userId'].unique()[:5]

all_recommendations = []

for user_id in sample_users:
    print(f"\n{'='*80}")
    print(f"RECOMMENDATIONS FOR USER {user_id}")
    print(f"{'='*80}")
    
    # Get user's rating history
    user_history = ratings_filtered[ratings_filtered['userId'] == user_id].merge(
        movies[['movieId', 'title', 'genres']], on='movieId'
    ).sort_values('rating', ascending=False).head(5)
    
    print("\nUser's Top Rated Movies:")
    print(user_history[['title', 'genres', 'rating']].to_string(index=False))
    
    # Get hybrid recommendations
    recs = get_hybrid_recommendations(user_id, top_n=10, alpha=0.7)
    
    print("\nTop 10 Hybrid Recommendations:")
    print(recs[['title', 'genres', 'hybrid_score']].to_string(index=False))
    
    # Store for later
    recs['userId'] = user_id
    all_recommendations.append(recs)

# Combine all recommendations
recommendations_df = pd.concat(all_recommendations, ignore_index=True)
print(f"\n\nGenerated {len(recommendations_df)} total recommendations for {len(sample_users)} users")

## 12. Prediction Error Analysis

In [ ]:
# Analyze prediction errors
errors_svd = [(pred.uid, pred.iid, pred.r_ui, pred.est, pred.est - pred.r_ui) 
              for pred in predictions_svd]
errors_df = pd.DataFrame(errors_svd, columns=['userId', 'movieId', 'actual', 'predicted', 'error'])

# Error statistics
print("Error Analysis for SVD Model:")
print("="*80)
print(f"Mean Error: {errors_df['error'].mean():.4f}")
print(f"Median Error: {errors_df['error'].median():.4f}")
print(f"Std Dev: {errors_df['error'].std():.4f}")
print(f"Min Error: {errors_df['error'].min():.4f}")
print(f"Max Error: {errors_df['error'].max():.4f}")

In [ ]:
# Visualize errors
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Error distribution
axes[0, 0].hist(errors_df['error'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[0, 0].set_xlabel('Prediction Error', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Distribution of Prediction Errors', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# Actual vs Predicted
axes[0, 1].scatter(errors_df['actual'], errors_df['predicted'], alpha=0.3, s=20)
axes[0, 1].plot([errors_df['actual'].min(), errors_df['actual'].max()], 
                [errors_df['actual'].min(), errors_df['actual'].max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[0, 1].set_xlabel('Actual Rating', fontsize=12)
axes[0, 1].set_ylabel('Predicted Rating', fontsize=12)
axes[0, 1].set_title('Actual vs Predicted Ratings', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Error by actual rating
error_by_rating = errors_df.groupby('actual')['error'].agg(['mean', 'std'])
axes[1, 0].bar(error_by_rating.index, error_by_rating['mean'], 
               yerr=error_by_rating['std'], edgecolor='black', alpha=0.7, capsize=5)
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Actual Rating', fontsize=12)
axes[1, 0].set_ylabel('Mean Prediction Error', fontsize=12)
axes[1, 0].set_title('Prediction Error by Actual Rating', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Absolute error by actual rating
abs_error_by_rating = errors_df.copy()
abs_error_by_rating['abs_error'] = abs_error_by_rating['error'].abs()
abs_error_by_rating.groupby('actual')['abs_error'].mean().plot(kind='bar', ax=axes[1, 1], 
                                                                 edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].set_xlabel('Actual Rating', fontsize=12)
axes[1, 1].set_ylabel('Mean Absolute Error', fontsize=12)
axes[1, 1].set_title('Absolute Error by Actual Rating', fontsize=14, fontweight='bold')
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=0)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Save Results

In [ ]:
# Save recommendations
recommendations_path = '../../../data/outputs/movie_recommendations.csv'
recommendations_df.to_csv(recommendations_path, index=False)
print(f"Recommendations saved to: {recommendations_path}")

# Save model comparison
comparison_path = '../../../data/outputs/recommendation_model_comparison.csv'
comparison_df.to_csv(comparison_path, index=False)
print(f"Model comparison saved to: {comparison_path}")

# Save error analysis
errors_path = '../../../data/outputs/prediction_errors.csv'
errors_df.head(1000).to_csv(errors_path, index=False)
print(f"Error analysis (sample) saved to: {errors_path}")

# Save movie statistics
movie_stats_path = '../../../data/outputs/movie_statistics.csv'
movie_stats.to_csv(movie_stats_path, index=False)
print(f"Movie statistics saved to: {movie_stats_path}")

print("\nAll results saved successfully!")

## 14. Summary and Key Findings

In [ ]:
print("="*80)
print("SESSION 8 SUMMARY: RECOMMENDATION SYSTEMS")
print("="*80)

print("\n1. DATASET OVERVIEW")
print(f"   - Total movies: {len(movies):,}")
print(f"   - Total ratings: {len(ratings):,}")
print(f"   - Unique users: {ratings['userId'].nunique():,}")
print(f"   - Average rating: {ratings['rating'].mean():.2f}")
print(f"   - Sparsity: {(1 - len(ratings) / (ratings['userId'].nunique() * ratings['movieId'].nunique())) * 100:.2f}%")

print("\n2. RECOMMENDATION APPROACHES IMPLEMENTED")
print("   a) User-Based Collaborative Filtering")
print("      - Finds similar users based on rating patterns")
print("      - Recommends items liked by similar users")
print(f"      - RMSE: {rmse_user:.4f}, MAE: {mae_user:.4f}")

print("\n   b) Item-Based Collaborative Filtering")
print("      - Finds similar items based on user ratings")
print("      - Recommends items similar to user's preferences")
print(f"      - RMSE: {rmse_item:.4f}, MAE: {mae_item:.4f}")

print("\n   c) Matrix Factorization (SVD)")
print("      - Decomposes user-item matrix into latent factors")
print("      - Captures hidden patterns in user preferences")
print(f"      - RMSE: {rmse_svd:.4f}, MAE: {mae_svd:.4f}")

print("\n   d) Content-Based Filtering")
print("      - Uses movie genres for similarity")
print("      - TF-IDF vectorization of genre information")
print("      - Cosine similarity for recommendations")

print("\n   e) Hybrid Approach")
print("      - Combines collaborative and content-based")
print("      - Weighted combination (alpha=0.7 for CF)")
print("      - Balances accuracy and diversity")

print("\n3. BEST PERFORMING MODEL")
best_model = comparison_df.loc[comparison_df['RMSE'].idxmin()]
print(f"   - Model: {best_model['Method']}")
print(f"   - RMSE: {best_model['RMSE']:.4f}")
print(f"   - MAE: {best_model['MAE']:.4f}")

print("\n4. EVALUATION METRICS")
print("   - RMSE (Root Mean Squared Error): Penalizes large errors")
print("   - MAE (Mean Absolute Error): Average prediction error")
print("   - Precision@K: Accuracy of top-K recommendations")
print("   - Cross-validation: 5-fold CV for robustness")

print("\n5. KEY INSIGHTS")
print("   - SVD generally outperforms KNN-based methods")
print("   - Hybrid approach provides more diverse recommendations")
print("   - Cold start problem addressed with content-based component")
print("   - High sparsity in data requires robust algorithms")

print("\n6. BUSINESS RECOMMENDATIONS")
print("   - Deploy hybrid system for balanced recommendations")
print("   - Use SVD for accurate rating predictions")
print("   - Implement content-based for new users/items")
print("   - Regular model retraining with new data")
print("   - A/B testing for recommendation strategies")

print("\n7. FILES GENERATED")
print("   - movie_recommendations.csv: Personalized recommendations")
print("   - recommendation_model_comparison.csv: Performance metrics")
print("   - prediction_errors.csv: Error analysis data")
print("   - movie_statistics.csv: Aggregated movie stats")

print("\n" + "="*80)
print("Recommendation system complete! Ready for production deployment.")
print("="*80)